### Test with a 100 _Arabidopsis_ genes and default mapper values

**Imort useful packages / modules**

In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ML tools
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

# For output display
from IPython.display import IFrame

# If running locally, set current directory as projdir
projdir = '.'

**Mount Google Drive**

Run the next cell __only if you're planning to run this notebook on Google Colab__. If you are running this notebook locally, comment it out.

To run the notebook in Google Colab, it is best if you upload data and script files to a folder in Google Drive and access them from there. We already have a shared Google Drive folder named `PlantsAndPython-2021-10-22` that contains all the required data and script files.
Run the next code cell to mount the drive and make the files accessible. We will define the shared folder as our project directory `projdir`.

If Google drive is not already mounted, running the cell will produce a link. Click on the link, follow the prompts to log in to your google account, and copy the text string generated at the end. Paste the text string in the box below and press `Enter`.

In [ ]:
# # Only if running in Google Colab..!!
# # DO NOT run this cell if running locally - simply comment it out.
# from google.colab import drive
# drive.mount('/content/gdrive/')

# projdir = '/content/gdrive/MyDrive/PlantsAndPython-2021-10-22'
# sys.path.append(projdir)

In [2]:
# Make sure your last path is the one were you have this script and your data
sys.path

['/001/usuarios/villasen/PlantsPython/ClassProject/Mapper',
 '/001/usuarios/villasen/anaconda3/lib/python37.zip',
 '/001/usuarios/villasen/anaconda3/lib/python3.7',
 '/001/usuarios/villasen/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/001/usuarios/villasen/.local/lib/python3.7/site-packages',
 '/001/usuarios/villasen/anaconda3/lib/python3.7/site-packages',
 '/001/usuarios/villasen/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/001/usuarios/villasen/.ipython']

In [3]:
# import helper_functions
from helper_functions import loaddata
from helper_functions import colorscale_from_matplotlib_cmap

# import lense function
from lenses import fsga_transform

# keppler mapper
import kmapper as km

#### Visualizing subsets of metadata and/or data of the *Arabidopsis* class project: 
Which one will be more useful to plot a preliminary mapper graph?



 * **BW**: subsetted by Brandon Webster
 * **RW**: subsetted by Robin Waterman
 * **KD**: subsetted by Kara Dobson
 * **SZ**: subsetted by Sophia Zorrilla
 
 
 *Note: there're probably more subsets hidden among the discord groups*

In [5]:
BW = pd.read_csv('./Databasemetadata.csv', encoding= 'unicode_escape') 
# I used encoding = 'unicode_escape' as a solution
# to an error I otherwise had: 'utf-8' codec can't decode 
# byte 0xea in position 11: invalid continuation byte
BW.head()

,Sample,Project,SampleName,PMID,Genotype,Ecotype,Tissue,TotalReads,UniqueMappedRate,ReleaseDate
0,DRX007662,PRJDB2180,Arabidopsis WT-Col mRNA_seq,/,wild type,Col-0,/,30664389,86.20%,2014/4/2
1,DRX007663,PRJDB2180,Arabidopsis ibm1-4 mRNA_seq,/,ibm1-4,Col-0,/,38551905,91.10%,2014/4/2
2,DRX007664,PRJDB2180,Arabidopsis ibm2-2 mRNA_seq,/,ibm2-2,Col-0,/,37223057,83.40%,2014/4/2
3,DRX014481,PRJDB1593,Y1,/,wild type,/,root,95012910,89.80%,2016/2/5
4,DRX014482,PRJDB1593,Y2,/,wild type,/,root,163269003,92.90%,2016/2/5


In [6]:
RW = pd.read_csv("./result_table.csv", encoding= 'unicode_escape')
RW.head()

,Sample,SampleName,AT1G22630,AT1G22620,AT1G22610,Tissue,Ecotype,Genotype,Treatment,Project,TotalReads,UniqueMappedRatio,ReleaseDate
0,DRX007662,Arabidopsis WT-Col mRNA_seq,37.66,28.68,0.0,--,Col-0,wild type,--,PRJDB2180,30664389,0.8615,4/2/2014
1,DRX007663,Arabidopsis ibm1-4 mRNA_seq,35.22,30.74,0.0,--,Col-0,ibm1-4,--,PRJDB2180,38551905,0.9114,4/2/2014
2,DRX007664,Arabidopsis ibm2-2 mRNA_seq,33.39,28.81,0.0,--,Col-0,ibm2-2,--,PRJDB2180,37223057,0.8343,4/2/2014
3,DRX014481,Y1,8.22,21.83,0.0,root,--,wild type,--,PRJDB1593,95012910,0.8982,2/5/2016
4,DRX014482,Y2,8.53,26.05,0.0,root,--,wild type,--,PRJDB1593,163269003,0.9293,2/5/2016


In [7]:
KD = pd.read_csv("./data_subset.csv")
KD.head()

,Unnamed: 0,Sample,SampleName,AT1G22630,AT1G22620,AT1G22610,Tissue,Ecotype,Genotype,Treatment,Project,TotalReads,UniqueMappedRatio,ReleaseDate
0,17117,GSM3178804,urt1 rep1 dataset #2,0.00,0.00,0.0,seedlings,--,urt1,--,PRJNA475117,1039688,0.2002,10/8/2018
1,2820,GSM1507902,Te ctrl rep3,61.64,29.59,0.0,rosette leaf,Te,Te,control,PRJNA261430,24510726,0.8412,9/22/2014
2,16447,GSM2915887,BA_2,19.16,36.65,0.0,seedlings,Col-0,--,BA,PRJNA429126,21775880,0.9044,10/1/2018
3,12342,SRX2899193,Col0_margin_rep2,95.46,23.76,0.0,leaves,Col-0,Col-0,--,PRJNA389787,10077394,0.7901,12/6/2017
4,159,DRX099518,DRS050614,0.00,15.69,0.0,root,Col-0,wild type,Mock,PRJDB6370,3990947,0.9492,9/28/2017


In [8]:
SZ = pd.read_csv("./result_table_100.csv")
SZ.head()

,Sample,SampleName,AT1G06190,AT1G59830,AT4G05510,AT5G45573,AT1G18340,AT4G13770,AT1G05520,AT1G48580,...,AT5G61500,AT5G00910,Tissue,Ecotype,Genotype,Treatment,Project,TotalReads,UniqueMappedRatio,ReleaseDate
0,DRX007662,Arabidopsis WT-Col mRNA_seq,42.03,38.25,0.03,0.0,16.85,15.03,32.44,8.81,...,46.68,0.89,--,Col-0,wild type,--,PRJDB2180,30664389,0.8615,2014/4/2
1,DRX007663,Arabidopsis ibm1-4 mRNA_seq,51.37,35.05,0.00,0.0,18.37,47.83,32.76,6.75,...,46.79,0.00,--,Col-0,ibm1-4,--,PRJDB2180,38551905,0.9114,2014/4/2
2,DRX007664,Arabidopsis ibm2-2 mRNA_seq,45.71,34.51,0.02,0.0,17.35,26.90,34.37,7.66,...,46.82,0.29,--,Col-0,ibm2-2,--,PRJDB2180,37223057,0.8343,2014/4/2
3,DRX014481,Y1,31.10,45.58,0.00,0.0,29.47,51.06,26.40,7.32,...,57.13,0.54,root,--,wild type,--,PRJDB1593,95012910,0.8982,2016/2/5
4,DRX014482,Y2,29.71,40.16,0.00,0.0,26.99,46.15,31.74,9.03,...,49.69,0.38,root,--,wild type,--,PRJDB1593,163269003,0.9293,2016/2/5


**Data**

We need to define as our dataframe `df` the database that contains gene columns (cells with gene expression values) and factor columns (ex. stress, tissue type, etc.). 

For this we'll eventually have to merge metadata (the factor columns) with the data (gene expression columns).

### Test with a 100 genes and default mapper values

In [98]:
df = SZ # set dataframe to be Sophia Zorrilla data subset (the one with 100 genes)
# df = SZ.iloc[0:5000,:]
print("rows, columns =", df.shape)
print("number of elements =", df.size)

rows, columns = (20068, 110)
number of elements = 2207480


In [105]:
genes = list(df.columns[2:102]) # create list with the gene names
genes

['AT1G06190',
 'AT1G59830',
 'AT4G05510',
 'AT5G45573',
 'AT1G18340',
 'AT4G13770',
 'AT1G05520',
 'AT1G48580',
 'AT1G11730',
 'AT5G23510',
 'AT5G27885',
 'AT5G27345',
 'AT1G15860',
 'AT1G07993',
 'AT2G07705',
 'AT3G13390',
 'AT5G02970',
 'AT1G08827',
 'AT2G33230',
 'AT1G30580',
 'AT3G29615',
 'AT3G05370',
 'AT3G03930',
 'AT2G21235',
 'AT4G03685',
 'AT5G53980',
 'AT5G54680',
 'AT3G23715',
 'AT3G08745',
 'AT2G13240',
 'AT5G40950',
 'AT5G10140',
 'AT5G23450',
 'AT1G77405',
 'AT1G65990',
 'AT2G07766',
 'AT5G65880',
 'AT2G21340',
 'AT3G33528',
 'AT5G43150',
 'AT4G08010',
 'AT1G25270',
 'AT1G06070',
 'AT1G29690',
 'AT4G37910',
 'AT5G22330',
 'AT3G60740',
 'AT2G35150',
 'AT5G08360',
 'AT3G03465',
 'AT2G06565',
 'AT2G26560',
 'AT1G50040',
 'AT1G07903',
 'AT4G12120',
 'AT2G06630',
 'AT1G08770',
 'AT2G34020',
 'AT4G23280',
 'AT3G55254',
 'AT1G07810',
 'AT2G33720',
 'AT5G05805',
 'AT2G38790',
 'AT5G04940',
 'AT3G02645',
 'AT5G49340',
 'AT1G48285',
 'AT1G16540',
 'AT2G06215',
 'AT1G28465',
 'AT5G

**Initialize a KeplerMapper object**

You can ignore the `nerve` part.

In [106]:
# Initialize mapper object
mymapper = km.KeplerMapper(verbose=1)

# Define Nerve
nerve = km.GraphNerve(min_intersection=1)

KeplerMapper(verbose=1)


**Define lens / filter function:**

Gene expression values

In [107]:
# Define lens
scaler = MinMaxScaler()
# residuals, idx_tr, idx_te = fsga_transform(df, orthos, filter_by_factor, filter_by_level) # For now, ignore Sourabh's function

lens = mymapper.project(df[genes], # our lens will be the genes expression values
                        projection='l2norm', 
                        scaler=scaler)

..Projecting on data shaped (20068, 100)

..Projecting data using: l2norm

..Scaling with: MinMaxScaler(copy=True, feature_range=(0, 1))



**Define cover:**

Overlap must be between 0 and 100. Intervals must be less than 130.

In [108]:
# Define cover
cubes, overlap = (100, 80) # cubes = intervals
cover = km.cover.Cover(n_cubes=cubes, perc_overlap=overlap/100.)

**Define clustering algorithm:**

DBSCAN with default parameters. Metric: correlation distance (1 - correlation) between a pair of gene expression profiles.

In [109]:
# Define clustering algorithm
clust_metric = 'correlation'
clusterer = DBSCAN(metric=clust_metric)

**Construct the mapper graph:**

Keep an eye on the number of hypercubes, nodes and edges reported by the algorithm. You can change the graph size by changing the cover parameters.

In [110]:
# Create mapper 'graph' with nodes, edges and meta-information.
graph = mymapper.map(lens=lens,
                     X=df[genes],
                     clusterer=clusterer,
                     cover=cover,
                     nerve=nerve,
                     precomputed=False,
                     remove_duplicate_nodes=True)

Mapping on data shaped (20068, 100) using lens shaped (20068, 1)

Creating 100 hypercubes.


/001/usuarios/villasen/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_base.py:814: RuntimeWarning: invalid value encountered in less_equal
  neigh_ind = [np.where(d <= radius)[0] for d in dist]
/001/usuarios/villasen/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_base.py:814: RuntimeWarning: invalid value encountered in less_equal
  neigh_ind = [np.where(d <= radius)[0] for d in dist]
/001/usuarios/villasen/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_base.py:814: RuntimeWarning: invalid value encountered in less_equal
  neigh_ind = [np.where(d <= radius)[0] for d in dist]
/001/usuarios/villasen/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_base.py:814: RuntimeWarning: invalid value encountered in less_equal
  neigh_ind = [np.where(d <= radius)[0] for d in dist]
/001/usuarios/villasen/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/_base.py:814: RuntimeWarning: invalid value encountered in less_equal
  neigh_ind = [np.where(d <= radius

Merged 4 duplicate nodes.

Number of nodes before merger: 8; after merger: 4


Created 2 edges and 4 nodes in 0:01:46.126121.


**Adding components to visualization**

Before we visualize the constructed mapper graph, we will add a couple of components to the visualization.<br>
First, we will color the nodes of the mapper graph using the specified factor (`color_by_factor`). The specified level (`color_by_level`) will be at one end of the colormap, all other levels will be at the other end. The node color is determined averaging the colors of all samples in the corresponding cluster.

In [ ]:
# # We are interested in three factors in particular: family, tissue type and stress type.

# factors = ['stress', 'tissue', 'family']
# levels = ['healthy', 'leaf', 'Poaceae']
# filter_by_factor, filter_by_level = ('family', 'Poaceae')
# color_by_factor, color_by_level = ('tissue', 'leaf')

In [ ]:
# # Color nodes by specified color_by_factor, color_by_level

# df[color_by_factor] = df[color_by_factor].astype('category')
# color_vec = np.asarray([0 if(val == color_by_level) else 1 for val in df[color_by_factor]])
# cscale = colorscale_from_matplotlib_cmap(plt.get_cmap('coolwarm'))

In [ ]:
# # show filter_by_factor levels in tooltip

# temp = ['({}, {})'.format(str(p[0]), str(p[1])) for p in zip(df[filter_by_factor], df[color_by_factor])]
# df['tooltips'] = temp

**Visualize the mapper graph**

Latly, we create the visualization, save it as an html file, and then load it into a frame.<br>
Alternatively, you can browse to the html file and open it in a separate browser window.

In [57]:
# # Specify file to save html output
# fname = 'FilterBy_{}_ColorBy_{}_Cubes_{}_Overlap_{}.html'.format(filter_by_factor,
#                                                               color_by_factor,
#                                                               cubes,
#                                                               overlap)
# figtitle = 'Lens: {} : {}, Color by {} : {}, # Intervals {}, overlap {}'.format(filter_by_factor,
#                                                                                 filter_by_level,
#                                                                                 color_by_factor,
#                                                                                 color_by_level,
#                                                                                 cubes, overlap/100.0)

fname = '100genesMapper.html' #set temporary name for the html mapper graph file
figtitle = '100 genes Mapper' #set name for the mapper graph

fpath = projdir + '/' + fname
# Create visualization and save to specified file
_ = mymapper.visualize(graph,
                       path_html=fpath,
                       title=figtitle)
                      #  color_values=color_vec,
                      #  color_function_name=color_by_factor,
                      #  colorscale=cscale,
                      #  custom_tooltips=df['tooltips'])

# Load the html output file
IFrame(src=fpath, width=1000, height=800)

Wrote visualization to: ./100genesMapper.html
